# LondonAir sites - subway stations mapping

# 0. Set-up

In [1]:
## import liabrairies
import pandas as pd
import numpy as np
import re
import requests
import json

# 1. Functions

In [2]:
## Function to get the closest site based on coordinates

def closest_site(df_sites, station_coordiantes):

    df_test_1 = df_sites.copy()
    df_test_1[['latitude', 'longitude']] = df_test_1[['latitude', 'longitude']].apply(pd.to_numeric)

    lat_val = station_coordiantes['Latitude']
    long_val = station_coordiantes['Longitude']

    df_test_1['Latitute_diff'] = (df_test_1['latitude']-lat_val).abs()
    df_test_1['Longitude_diff'] = (df_test_1['longitude']-long_val).abs()

    df_test_1['loc_diff'] = df_test_1['Latitute_diff'] + df_test_1['Longitude_diff']

    index_min = df_test_1['loc_diff'].idxmin()
    station_min = df_test_1.iloc[index_min]['site_code']

    return station_min

In [3]:
## Function to put the results of the mapping back into a source dataframe

def map_sites(df_stations_coordinates, df_sites_map):
    df_update = df_stations_coordinates.copy()
    df_update['closest_site'] = np.NaN
    
    for i in range(len(df_update)):
        x = closest_site(df_sites_map, df_update.loc[i])
        df_update['closest_site'].iloc._setitem_with_indexer(i, x)
        
    return df_update

# 2. Execution

In [4]:
# Files location
file_loc = 'C:\\Users\\user\\Documents_C\\Python\\Jupyter\\05. Data engineering\\Group_Project\\'

# Import data for each data set
df_stations_original = pd.read_csv((file_loc +'station_data.csv'))
df_sites = pd.read_csv((file_loc + 'London_Air_sites_information.csv'))

In [5]:
df_stations = df_stations_original[['Station', 'Latitude', 'Longitude']].copy()
df_stations = df_stations.drop_duplicates(subset='Station')
df_stations = df_stations.reset_index(drop = True)
df_stations

,Station,Latitude,Longitude
0,Abbey Road,51.531952,0.003723
1,Abbey Wood,51.490784,0.120272
2,Acton Central,51.508758,-0.263430
3,Acton Main Line,51.516887,-0.267690
4,Acton Town,51.503071,-0.280303
...,...,...,...
638,Woodside Park,51.617868,-0.185426
639,Woolwich,51.491578,0.071819
640,Woolwich Arsenal,51.489907,0.069194
641,Woolwich Dockyard,51.491108,0.054612


In [6]:
df_sites_map = df_sites[['site_code', 'latitude', 'longitude']].copy()
df_sites_map

,site_code,latitude,longitude
0,BG1,51.563752,0.177891
1,BG2,51.529389,0.132857
2,BX2,51.490610,0.158914
3,BX0,51.490610,0.158914
4,BQ7,51.494649,0.137279
...,...,...,...
101,MY1,51.522540,-0.154590
102,MY7,51.522540,-0.154590
103,WM6,51.513929,-0.152793
104,WMB,51.516066,-0.135164


In [7]:
station_site_map = map_sites(df_stations, df_sites_map).copy()
station_site_map.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

<ipython-input-3-525b06d5f628>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_update['closest_site'].iloc._setitem_with_indexer(i, x)


# 3. Output

In [8]:
station_site_map

,Station,closest_site
0,Abbey Road,TH4
1,Abbey Wood,BQ7
2,Acton Central,EA8
3,Acton Main Line,EI8
4,Acton Town,EI3
...,...,...
638,Woodside Park,EN5
639,Woolwich,GN0
640,Woolwich Arsenal,GN0
641,Woolwich Dockyard,GN0
